# libraries

In [3]:
import cometspy as c
import os
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import cobra
import time, random

In [4]:
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)
print("Cobra version:", cobra.__version__)

Pandas version: 2.0.0
NumPy version: 1.26.4
Cobra version: 0.29.1


# data - models(20) and diet

In [33]:
#diet
diet = pd.read_csv('Magnusdottir_2017_TableS12_diets.csv')
diet.head()

,metabolite_ID,Exchange_reaction,metabolite_name,western,high_fiber
0,arab_L,EX_arab_L(e),L-arabinose,0.178783,0.047368
1,cellb,EX_cellb(e),Cellobiose,0.074493,0.019737
2,drib,EX_drib(e),2-deoxy-D-ribose,0.178783,0.047368
3,fru,EX_fru(e),D-Fructose,0.148986,0.039474
4,fuc_L,EX_fuc_L(e),L-fucose,0.148986,0.039474


In [34]:
diet_dict = dict(zip(diet['metabolite_ID'], diet['western'])) #initial diet dictionary

# models

In [7]:
# model dict to store the models

models_dir = "models20"
model_files = sorted(os.listdir(models_dir))

models_meta = {} # id: name
models = {} # id: model
#comets_models = {} # id: comets model

for i in range(20): #change range according to the number of models
    model_name = model_files[i].split(".")[0]
    model_path = os.path.join(models_dir, model_files[i])
    models_meta[i+1] = model_name #update metadata
    print('loading model', i+1,'of 20 models...')

    model = cobra.io.read_sbml_model(model_path) 
    models[i+1] = model #update models dict
    #comets_model = c.model(model)

print('Done loading the models!')

loading model 1 of 20 models...
loading model 2 of 20 models...
loading model 3 of 20 models...
loading model 4 of 20 models...
loading model 5 of 20 models...
loading model 6 of 20 models...
loading model 7 of 20 models...
loading model 8 of 20 models...
loading model 9 of 20 models...
loading model 10 of 20 models...
loading model 11 of 20 models...
loading model 12 of 20 models...
loading model 13 of 20 models...
loading model 14 of 20 models...
loading model 15 of 20 models...
loading model 16 of 20 models...
loading model 17 of 20 models...
loading model 18 of 20 models...
loading model 19 of 20 models...
loading model 20 of 20 models...
Done loading the models!


# simulations

## monoculture parameters

In [35]:
#parameters - chemostat condition
#monoculture part1
dilution_rate = 0.017
mono_params = c.params()

mono_params.set_param("defaultVmax", 20.) #vmax = 20 mmol gDW-1h-1 for all metabolites
mono_params.set_param("defaultKm", 0.05) #km = 0.05 mmol

mono_params.set_param("spaceWidth", 0.1) #0.1 cm
mono_params.set_param('maxSpaceBiomass', 10.) #gDW - 10per species

mono_params.set_param('deathRate', dilution_rate)
mono_params.set_param('metaboliteDilutionRate', dilution_rate)   
mono_params.set_param('batchDilution', False)

#save media and flux data
mono_params.set_param('writeFluxLog', True)
mono_params.set_param('writeMediaLog', True)
mono_params.set_param('FluxLogRate', 10)
mono_params.set_param('MediaLogRate', 10)
#mono_params.set_param('writeBiomassLog', True) 
mono_params.set_param('writeTotalBiomassLog', True) #save to output if True
mono_params.set_param('totalBiomassLogRate', 1) #save to output every 1 timesteps
#mono_params.show_params()


mono_params.set_param('timeStep', 0.1) #hours - The amount of time between two consecutive simulation updates
mono_params.set_param('maxCycles', 1050) #three days #duration of simulation in timesteps - number of simulation cycles #30 hours

## monoculture simulation

### monoculture

In [36]:
initial_pop = 0.01 #initial population of the model
dilution_rate = 0.017 # /hr

biomass_dict = {} #initialize biomass dict for storing timeseries biomass data - id:timeseries biomass data
#media_dict = {} #initialize media dict for storing timeseries media data - id:timeseries media data
biomass_df = pd.DataFrame() #initialize dataframe to store biomass data
#media_df = pd.DataFrame() #initialize dataframe to store media data

#parameters update part2
mono_params.set_param('deathRate', dilution_rate)
mono_params.set_param('metaboliteDilutionRate', dilution_rate)

#time watch
start_time = time.time()

# create layout and run monoculture simulations
for i in range(20):
    model = models[i+1]
    #make comets model
    comets_model = c.model(model)
    #set model id
    comets_model.id = 'model' + str(i+1)
    #set initial population and open exchanges
    comets_model.initial_pop = [0,0,initial_pop] # x, y, gDW
    comets_model.open_exchanges()

    #create layout
    print('Constructing layout for model', i+1, '...')
    mono_layout = c.layout(comets_model)
    #print(mono_layout.get_model_ids())

    ## NOTE: cobrapy media are set using exchange reaction IDs, whereas COMETS media are set using metabolite ids

    # update log file names
    mono_params.set_param('FluxLogName', f'model{i+1}_mono_fluxlog.txt')
    mono_params.set_param('MediaLogName', f'model{i+1}_mono_medialog.txt')
    mono_params.set_param('BiomassLogName', f'model{i+1}_mono_biomasslog.txt')
    
    #set diet
    for met in diet_dict:
        metabolite = (met+'[e]')
        conc = diet_dict[met]
        #print(metabolite, conc)
        if metabolite in mono_layout.media['metabolite'].values:
            mono_layout.set_specific_metabolite(metabolite, conc) #todo: find a way to suppress warnings

    #set media refresh rate - chemostat conditions
    for met in mono_layout.media['metabolite']:
        initconc = mono_layout.media[mono_layout.media['metabolite'] == met]['init_amount'].values[0]
        refreshrate = initconc*dilution_rate
        mono_layout.set_specific_refresh(met, refreshrate)

    mono_simulation = c.comets(mono_layout, mono_params)
    mono_simulation.run()

    #save biomass data
    biomass_dict[i+1] = mono_simulation.total_biomass
    #media_dict[i] = simulation.get_metabolite_time_series()
    
    #save cycle indices to dataframe
    if i == 0:
        biomass_df['cycle'] = biomass_dict[i+1]['cycle']
        #media_df['cycle'] = simulation.get_cycle_indices()

    #save biomass data to dataframe
    biomass_df['model'+str(i+1)] = biomass_dict[i+1][f'model{i+1}']

    print('model', i+1, 'done!')
    print('\n\n ************************************ \n\n')
    time.sleep(2) #sleep for 5 seconds to avoid overloading the server

end_time = time.time()
print('Monoculture simulations done!')
print('Time taken:', end_time-start_time, 'seconds')

c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.


Constructing layout for model 1 ...

Running COMETS simulation ...

Debug Here ...
Done!
model 1 done!


 ************************************ 




c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.


Constructing layout for model 2 ...

Running COMETS simulation ...

Debug Here ...
Done!
model 2 done!


 ************************************ 




c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.


...

Constructing layout for model 18 ...

Running COMETS simulation ...

Debug Here ...
Done!
model 18 done!


 ************************************ 




c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.


Constructing layout for model 19 ...

Running COMETS simulation ...

Debug Here ...
Done!
model 19 done!


 ************************************ 




c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.


Constructing layout for model 20 ...

Running COMETS simulation ...

Debug Here ...
Done!
model 20 done!


 ************************************ 


Monoculture simulations done!
Time taken: 585.7643208503723 seconds


In [37]:
biomass_df.head()

,cycle,model1,model2,model3,model4,model5,model6,model7,model8,model9,...,model11,model12,model13,model14,model15,model16,model17,model18,model19,model20
0,0,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,...,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000
1,1,0.020658,0.039907,0.027220,0.009983,0.031055,0.027066,0.046639,0.009983,0.050755,...,0.009983,0.009983,0.051007,0.009983,0.027066,0.017027,0.027220,0.009983,0.041911,0.009983
2,2,0.042672,0.141765,0.074091,0.009966,0.089126,0.073258,0.197823,0.009966,0.223298,...,0.009966,0.009966,0.214958,0.009966,0.073258,0.028990,0.074091,0.009966,0.164090,0.009966
3,3,0.072200,0.304329,0.154102,0.009949,0.152890,0.108349,0.435538,0.009949,0.524790,...,0.009949,0.009949,0.429183,0.009949,0.198282,0.043163,0.171657,0.009949,0.367068,0.009949
4,4,0.094785,0.421327,0.229189,0.009932,0.209833,0.111571,0.692677,0.009932,0.789469,...,0.009932,0.009932,0.629208,0.009932,0.256246,0.054924,0.241824,0.009932,0.467569,0.009932


In [38]:
biomass_df.to_csv('biomass_df_monoculture20.csv', index=False)

## community parameters 

In [39]:
# community parameters
com_params = c.params()
#random.seed(33)

initial_pop = 0.01  # Initial biomass of newly introduced species
dilution_rate = 0.017  # Chemostat dilution rate

com_params.set_param("defaultVmax", 20.) #vmax = 20 mmol gDW-1h-1 for all metabolites
com_params.set_param("defaultKm", 0.05) #km = 0.05 mmol

com_params.set_param("spaceWidth", 0.1) #0.1 cm
com_params.set_param('maxSpaceBiomass', 10.) #gDW - 10per species

#save media and flux data
com_params.set_param('writeFluxLog', True)
com_params.set_param('writeMediaLog', True)
com_params.set_param('FluxLogRate', 40)
com_params.set_param('MediaLogRate', 40)
#com_params.set_param('writeBiomassLog', True) 
com_params.set_param('writeTotalBiomassLog', True) #save to output if True
com_params.set_param('totalBiomassLogRate', 1) #save to output every 1 timesteps
#com_params.show_params()

com_params.set_param('timeStep', 0.1) #hours - The amount of time between two consecutive simulation updates
com_params.set_param('maxCycles', 50) #three days #duration of simulation in timesteps - number of simulation cycles #30 hours

com_params.set_param('randomSeed', 33)
com_params.set_param('deathRate', dilution_rate)
com_params.set_param('metaboliteDilutionRate', dilution_rate)   
com_params.set_param('batchDilution', False)

#time watch
com_params.set_param('showCycleCount', True)
com_params.set_param('showCycleTime', True)

## community functions

In [40]:
# Function to set up a COMETS model for each species
def setup_comets_model(model, model_id, initial_pop): #cobra model, modelindex, initial population
    comets_model = c.model(model)
    comets_model.id = model_id
    comets_model.initial_pop = [0, 0, initial_pop]  # x, y, gDW
    comets_model.open_exchanges()
    return comets_model

# Function to set up the layout with diet and refresh rate
def setup_layout(models_list, diet_dict, dilution_rate):
    comets_models = []
    for i in models_list:
        model = models[i]
        model_id = 'model' + str(i)
        comets_model = setup_comets_model(model, model_id, initial_pop)
        comets_models.append(comets_model)
    #print('comets_models:', comets_models)

    layout = c.layout(comets_models)

    for met in diet_dict:
        metabolite = met + '[e]'
        conc = diet_dict[met]
        if metabolite in layout.media['metabolite'].values:
            layout.set_specific_metabolite(metabolite, conc)

    # Set media refresh rate (chemostat conditions)
    for met in layout.media['metabolite']:
        initconc = layout.media[layout.media['metabolite'] == met]['init_amount'].values[0]
        refreshrate = initconc * dilution_rate
        layout.set_specific_refresh(met, refreshrate)
        
    return layout

# Function to run simulation for a given layout and parameters
def run_simulation(layout, params):
    simulation = c.comets(layout, params)
    simulation.run()
    return simulation


# Function to randomly select species from the pool
def select_new_species(max_community_size, prev_biomass_dict):
    species_list = list(range(1,max_community_size+1))
    
    #Remove species that are already present
    for i in prev_biomass_dict:
        print(prev_biomass_dict[i])
        if prev_biomass_dict[i] > 10**-10:
            sp = int(i[5:])
            species_list.remove(sp)

    sp_idx = random.choice(species_list)
    return sp_idx

def setup_invader_layout(invader_idx, diet_dict, prev_biomass_dict, dilution_rate, updated_diet_dict):
    invader_cmodel = setup_comets_model(models[invader_idx], f'model{invader_idx}', initial_pop)
    layout = c.layout([invader_cmodel])
    for i in prev_biomass_dict:
        sp_id = int(i[5:])
        biomass = prev_biomass_dict[i]
        comets_model = setup_comets_model(models[sp_id], i, biomass)
        layout.add_model(comets_model)
    for met in diet_dict:
        metabolite = met + '[e]'
        conc = diet_dict[met]
        if metabolite in layout.media['metabolite'].values:
            layout.set_specific_metabolite(metabolite, conc)
    
    # Update media according to previous simulation's results and set media refresh rate (chemostat conditions)
    for met in layout.media['metabolite']:
        if met in updated_diet_dict:
            conc = updated_diet_dict[met]
            layout.set_specific_metabolite(met, conc)
        initconc = layout.media[layout.media['metabolite'] == met]['init_amount'].values[0]
        refreshrate = initconc * dilution_rate
        layout.set_specific_refresh(met, refreshrate)
    return layout

In [60]:
list(community_layout.media['metabolite'])

['15dap[e]',
 '34dhpha[e]',
 '34dhphe[e]',
 '3hphac[e]',
 '3mop[e]',
 '4abz[e]',
 '4hbz[e]',
 '5htrp[e]',
 '5mta[e]',
 'Lcyst[e]',
 'ac[e]',
 'acald[e]',
 'acgam[e]',
 'actn_R[e]',
 'adn[e]',
 'adocbl[e]',
 'akg[e]',
 'ala_D[e]',
 'ala_L[e]',
 'alaasp[e]',
 'alagln[e]',
 'alaglu[e]',
 'alagly[e]',
 'alahis[e]',
 'alaleu[e]',
 'alathr[e]',
 'alltn[e]',
 'arab_L[e]',
 'arbt[e]',
 'arg_L[e]',
 'asn_L[e]',
 'aso3[e]',
 'aso4[e]',
 'asp_L[e]',
 'biomass[c]',
 'btd_RR[e]',
 'butso3[e]',
 'ca2[e]',
 'cbl1[e]',
 'cbl2[e]',
 'cd2[e]',
 'cellb[e]',
 'cgly[e]',
 'chol[e]',
 'cit[e]',
 'cl[e]',
 'co2[e]',
 'cobalt2[e]',
 'csn[e]',
 'cu2[e]',
 'cynt[e]',
 'cys_L[e]',
 'cytd[e]',
 'dad_2[e]',
 'dcyt[e]',
 'dgsn[e]',
 'din[e]',
 'dopa[e]',
 'drib[e]',
 'duri[e]',
 'etha[e]',
 'ethso3[e]',
 'etoh[e]',
 'fe2[e]',
 'fe3[e]',
 'fecrm[e]',
 'for[e]',
 'fru[e]',
 'fum[e]',
 'gal[e]',
 'galctn_D[e]',
 'galt[e]',
 'galur[e]',
 'gam[e]',
 'gcald[e]',
 'glc_D[e]',
 'glcn[e]',
 'glcur[e]',
 'gln_L[e]',
 'glu_L[

#  evolution - 20x20

In [53]:
max_community_size = 20
species_dict = {} #initialize species dict to store species composition at each assembly sequence

for technical_replicate in range(1, 11):
    sp_id_list = []
    biomass_dict = {}
    simulation_dict = {}
    biomass_df = pd.DataFrame()

    for i in range(1, max_community_size+1):
        biomass_dict[i] = {}

    print('Starting technical replicate', technical_replicate, '...')

    for com_size in range(1, 21):

        print(f'Building community size of {com_size}...')

        if com_size == 1:
            sp_idx = random.choice(range(1, 21))
            sp_id_list.append(sp_idx)

            model = models[sp_idx]
            comets_model = setup_comets_model(model, f'model{sp_idx}', initial_pop)

            #layout
            community_layout = setup_layout([sp_idx], diet_dict, dilution_rate)

            #update params
            com_params.set_param('maxCycles', 50)  # Simulate for 6 hours
            com_params.set_param('timeStep', 0.1)

            #simulation
            simulation_dict[com_size] = run_simulation(community_layout, com_params)
            
            # Save biomass after 5 hours
            species_biomass = simulation_dict[com_size].total_biomass.loc[simulation_dict[com_size].total_biomass['cycle'] == 50, f'model{sp_idx}'].values[0]
            biomass_dict[com_size][f'model{sp_idx}'] = species_biomass
            print('First species:', sp_idx, 'biomass:', species_biomass)
            print('First species simulation done!')
            print('\n\n ************************************ \n\n')

        else:
            prev_biomass_dict = biomass_dict[com_size - 1]
            inv_idx = select_new_species(20, prev_biomass_dict)
            sp_id_list.append(inv_idx)
            print('Adding model:', inv_idx, 'to the community...')

            #get previous media concentrations
            media_df_prev_sim = simulation_dict[com_size-1].media.loc[simulation_dict[com_size-1].media['cycle'] == 50].reset_index(drop=True)
            updated_diet_dict = dict(zip(media_df_prev_sim['metabolite'], media_df_prev_sim['conc_mmol']))

            updated_comlayout = setup_invader_layout(inv_idx, diet_dict, prev_biomass_dict, dilution_rate, updated_diet_dict)

            
            # Run simulation for 5 hours
            simulation_dict[com_size] = run_simulation(updated_comlayout, com_params)

            for i in sp_id_list:
                species_biomass = simulation_dict[com_size].total_biomass.loc[simulation_dict[com_size].total_biomass['cycle'] == 50, f'model{i}'].values[0]
                biomass_dict[com_size][f'model{i}'] = species_biomass

            print(f'Simulation of community size {com_size} done!')
            print('\n\n ************************************ \n\n')

        
            if com_size == 20:
                print(f'Simulation of assembly seq done!')
                print('\n\n ************************************ \n\n')

    species_dict[technical_replicate] = sp_id_list

    for i in range(1, 21):  # Loop over the simulation_dict
        temp_df = simulation_dict[i].total_biomass.copy()  # Make a copy of the DataFrame for iteration i
        temp_df['com_size'] = i  # Add the 'seq_num' column to the DataFrame
        biomass_df = pd.concat([biomass_df,temp_df], ignore_index=True) 

    biomass_df.to_csv(f'results200x20/biomass_data_seq_invasion20x20_csize{com_size}_techrep{technical_replicate}.csv', index=False)
    print('Technical_replicate', technical_replicate, 'done!')


Starting technical replicate 1 ...
Building community size of 1...


c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.
c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.



Running COMETS simulation ...

Debug Here ...
Done!
First species: 14 biomass: 0.0091844584898
First species simulation done!


 ************************************ 


Building community size of 2...
0.0091844584898
Adding model: 20 to the community...


c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.
c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.



Running COMETS simulation ...

Debug Here ...
Done!
Simulation of community size 2 done!


 ************************************ 


Building community size of 3...
0.12167733025
0.17630189981
Adding model: 2 to the community...


...


Running COMETS simulation ...

Debug Here ...
Done!
Simulation of community size 18 done!


 ************************************ 


Building community size of 19...
0.90556800421
6.2740798929
1.2771816234
0.052748426673
0.32834047819
0.047643682486
0.019166655078
0.0039227362809
0.0042710588602
0.004650311028
0.0050632392026
0.0055128336724
0.0060023502512
0.0065353338554
0.007115644175
0.0077474836245
0.0084354277751
0.0091844584898
Adding model: 17 to the community...


c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.
c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.
c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.
c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in 


Running COMETS simulation ...

Debug Here ...
Done!
Simulation of community size 19 done!


 ************************************ 


Building community size of 20...
0.86721857156
6.953767877
1.1730221604
0.064050768117
0.39524242107
0.04375814241
0.017603534795
0.0036028208539
0.0039227362809
0.0042710588602
0.004650311028
0.0050632392026
0.0055128336724
0.0060023502512
0.0065353338554
0.007115644175
0.0077474836245
0.0084354277751
0.0091844584898
Adding model: 13 to the community...


c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.
c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.
c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self.objective = [int(self.reactions[self.reactions.
c:\Users\sabh\Desktop\comets\cometsenv\Lib\site-packages\cometspy\model.py:667: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in 


Running COMETS simulation ...

Debug Here ...
Done!
Simulation of community size 20 done!


 ************************************ 


Simulation of assembly seq done!


 ************************************ 


Technical_replicate 10 done!


In [49]:
for i in range(1, 21):  # Loop over the simulation_dict
    temp_df = simulation_dict[i].total_biomass.copy()  # Make a copy of the DataFrame for iteration i
    temp_df['com_size'] = i  # Add the 'seq_num' column to the DataFrame
    biomass_df = pd.concat([biomass_df,temp_df], ignore_index=True) 

In [51]:
biomass_df

,cycle,model5,com_size,model17,model7,model14,model16,model10,model6,model12,...,model4,model13,model8,model9,model20,model11,model1,model19,model3,model15
0,0,0.010000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.031055,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.089126,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.152890,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.209833,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,46,0.958289,20,3.122128,3.225855,0.002758,0.023165,0.387388,0.072008,0.003332,...,0.004300,0.004682,0.005098,0.005550,0.006043,0.006580,0.007164,0.007800,0.008493,0.009247
1016,47,0.957466,20,3.116986,3.220371,0.002753,0.023126,0.386729,0.071885,0.003326,...,0.004293,0.004674,0.005089,0.005541,0.006033,0.006569,0.007152,0.007787,0.008479,0.009231
1017,48,0.957949,20,3.112106,3.214896,0.002748,0.023087,0.386072,0.071763,0.003320,...,0.004286,0.004666,0.005080,0.005532,0.006023,0.006558,0.007140,0.007774,0.008464,0.009216
1018,49,0.959151,20,3.107218,3.209431,0.002744,0.023047,0.385415,0.071641,0.003315,...,0.004278,0.004658,0.005072,0.005522,0.006013,0.006546,0.007128,0.007761,0.008450,0.009200


In [54]:
species_dict

{1: [14, 20, 2, 19, 13, 1, 15, 3, 12, 18, 10, 16, 17, 6, 5, 8, 9, 7, 4, 11],
 2: [1, 9, 18, 10, 3, 19, 17, 7, 20, 11, 15, 16, 12, 14, 8, 2, 4, 6, 5, 13],
 3: [8, 12, 13, 1, 15, 11, 16, 3, 7, 10, 14, 5, 4, 6, 20, 2, 19, 18, 9, 17],
 4: [4, 6, 7, 11, 15, 18, 13, 9, 2, 3, 10, 14, 20, 17, 1, 5, 8, 19, 16, 12],
 5: [13, 17, 16, 18, 10, 7, 6, 14, 4, 5, 19, 3, 12, 1, 9, 20, 8, 11, 2, 15],
 6: [15, 11, 8, 13, 19, 5, 14, 16, 12, 20, 18, 3, 6, 7, 2, 4, 10, 1, 17, 9],
 7: [8, 2, 14, 1, 15, 18, 19, 16, 17, 13, 11, 7, 6, 3, 5, 20, 9, 10, 4, 12],
 8: [6, 3, 13, 7, 11, 16, 2, 14, 18, 8, 1, 15, 5, 10, 20, 4, 17, 19, 9, 12],
 9: [6, 15, 5, 13, 4, 18, 16, 1, 9, 11, 3, 12, 17, 2, 19, 8, 10, 7, 20, 14],
 10: [19, 3, 4, 1, 5, 6, 7, 11, 9, 15, 12, 16, 18, 8, 10, 2, 20, 14, 17, 13]}